## 1. Inspecting request_responses.csv file
<p>Nabd App provide easy, fast and reliable way of communication for those who are in need for urgent advice on how to deal with a medical emergency. Trauma specialists call the first 60 minutes after a serious injury "the golden hour". As chances of survival are greatest if the patient gets proper treatment in that hour. This obviously won't replace the need for a doctor, but it helps until medical transport arrives. We are modelling responses to get insights about users' behaviors that will help us improve the app.</p>
<p>We generated a dummy dataset to test the model using []. Once we deploy the app we will link the model to the database to perform our analysis.</p>
<p>The data is stored in <code>datasets/request_responses.csv</code> and it is structured according to RFMTC marketing model (a variation of RFM). </p>
<p> RFM stands for Recency, Frequency and Monetary Value and it is commonly used in marketing for identifying your best customers. In our case, our customers are request responders.
RFMTC is a variation of the RFM model. Below is a description of what each column means in our dataset:</p>
<ul>
<li> R (Recency - time since the last response in hours) </li>
<li> F (Frequency - total number of responses)</li>
<li> M (Monetary - duration of call in minutes)</li>
<li>T (Time - months since the first response)</li>
<li> a binary variable representing whether he/she responded to a call in the last week (1 stands for responding; 0 stands for not responding)</li>
<ul>

In [ ]:
# Import pandas
import pandas as pd

# Read in dataset
responses = pd.read_csv('datasets/request_responses.csv')

# Print out the first rows of our dataset
responses.head()